In [1]:
%matplotlib inline

In [17]:
'''
Created on October 24, 2016

@author: philippschw

 https://developer.flightstats.com/api-docs/scheduledFlights/v1

'''
from __future__ import absolute_import

import os
import requests
import datetime
from pytz import timezone
from flightstats.flightaware_airports import AIRPORTS as FA_AIRPORTS

APPLICATION_ID = os.environ['Flightstats_ID']
APPLICATION_KEY = os.environ["FLIGTHSTATS_Key"]

def send_request(search_url):
    ("This is the first line of my text, " + 
"which will be joined to a second.")
    
    req_url = ("https://api.flightstats.com/flex/schedules/rest/v1/json/" +
    "{}?appId={}&appKey={}&codeType=IATA").format(search_url,APPLICATION_ID, APPLICATION_KEY)
    resp = requests.get(req_url)
    if resp.status_code == requests.codes.ok:  # @UndefinedVariable pylint:disable=no-member
        return resp.json()

def arrivals(from_airport, to_airport, arrival_date):
    """
        finds arrivals
        @from_airport The airport code (IATA) of the departure airport (required)
        @to_airport The airport code (IATA) of the arrival airport (required)
        @arrival_date arrival date (required)
    """
    search_url = ("from/{from_airport}/to/{to_airport}/arriving/" +
    "{arrival_year}/{arrival_month}/{arrival_day}").format(from_airport=from_airport,
                                                           to_airport=to_airport,
                                                           arrival_year=arrival_date.year,
                                                           arrival_month=arrival_date.month,
                                                           arrival_day=arrival_date.day)
    content = send_request(search_url)
    return content

def departures(from_airport, to_airport, departure_date):
    """
        finds departure
        @from_airport The airport code (IATA) of the departure airport (required)
        @to_airport The airport code (IATA) of the arrival airport (required)
        @departure_date departure date (required)
    """
    search_url = ("from/{from_airport}/to/{to_airport}/departing/"+
    "{departure_year}/{departure_month}/{departure_day}").format(from_airport=from_airport,
                                                                 to_airport=to_airport,
                                                                 departure_year=departure_date.year,
                                                                 departure_month=departure_date.month,
                                                                 departure_day=departure_date.day)
    content = send_request(search_url)
    return content

In [18]:
import pandas as pd
airports = pd.read_csv('airports.csv')

In [19]:
# airports= airports.set_index(["country"]).ix['Germany'].reset_index()
df_all = pd.DataFrame()

for airport in airports.code_iata.tolist(): 
    
    a = arrivals("FRA", airport, datetime.date(2016,10, 25))
    df = pd.DataFrame(a["scheduledFlights"])
#     print df
    df_all = pd.concat([df_all,df], axis=1)
    

In [20]:
len(df_all)

73

In [21]:
df_all.head()

,arrivalAirportFsCode,arrivalTime,carrierFsCode,codeshares,departureAirportFsCode,departureTerminal,departureTime,flightEquipmentIataCode,flightNumber,isCodeshare,...,flightEquipmentIataCode,flightNumber,isCodeshare,isWetlease,referenceCode,serviceClasses,serviceType,stops,trafficRestrictions,wetleaseOperatorFsCode
0,KEF,2016-10-25T15:35:00.000,FI,[],FRA,2,2016-10-25T14:00:00.000,757,521,False,...,EMJ,1380,False,True,1896-1513334--,"[F, J, Y]",J,0.0,[],CL
1,KEF,2016-10-25T14:20:00.000,WW*,"[{u'trafficRestrictions': [], u'serviceType': ...",FRA,2,2016-10-25T12:45:00.000,32S,761,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KEF,2016-10-25T14:20:00.000,W2,[],FRA,2,2016-10-25T12:45:00.000,32S,3761,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
